# Practical Project Crowberry Data Scientist
21.01.2021

In [23]:
import pandas as pd
import numpy as np

In [24]:
#pd.set_option('display.max_columns',None)
df = pd.read_excel("Data Scientist - Project.xlsx",header=0)

This notebook is based on the table with 8894 startup records and 45 property columns. Final startups should be by the following criteria: (There are probably more, depending on analytical results WIP) <br>
1. HQ Country are the Nordics (Denmark, Sweden, Norway, Finland, Iceland) with the direct consequence of HQ Region Europe
2. Growth stage: Pre-seed stage, Seed stage and A-Round (could be measured/obtained by the columns GROWTH STAGE and LAST ROUND)
3. Funding: If the company was already funded, the last funding (LAST FUNDING DATE) should not be older than 4 years. Either the startups is already independent or not active/ early in tech

There are three more conventions Crowberry focusses on:
- early in tech
- company first
- Opportunity in Inclusivity

These perspectives will be considered in the later and more detailed stage of this project.

## 1. HQ Country

In [25]:
countries = df['HQ COUNTRY'].unique()
nordics = ['Sweden', 'Norway', 'Iceland', 'Denmark', 'Finland']
df_filtered = df[df['HQ COUNTRY'].isin(nordics)]

df_filtered.reset_index();

Orginal table is reduced to a size if 8698 records by setting HQ COUNTRY to the nordics. Recheck of the new table:

In [26]:
df_filtered['HQ COUNTRY'].unique()

array(['Sweden', 'Finland', 'Denmark', 'Iceland', 'Norway'], dtype=object)

##### CHECK :-)

## 1. Transform column formats

### 1.1 VALUATION and VALUATION (EUR)

In [27]:
#Function to set average value from valuation interval (min + max)/2

def avg_valuation(cellValue):
    val = cellValue.split('-')
    return (float(val[1])+ float(val[0]))/2


#Parse both valuation columns and create maximum valuation column

valuationColumns = ['VALUATION', 'VALUATION (EUR)']

for column in valuationColumns:
    df_filtered.loc[:,column + ' AVG'] = df_filtered[column]
    df_filtered.loc[:,column + ' AVG'] = [avg_valuation(cell) if isinstance(cell, str) 
                                else cell for cell in df_filtered['VALUATION AVG']]

#df_filtered['VALUATION MAX']   

### 1.2 LAST FUNDING DATE, FIRST FUNDING DATE and VALUATION DATE

In [28]:
#Parsing and transforming date columns LAST FUNDING DATE, FIRST FUNDING DATE and VALUATION DATE to real date format

def parse_year_to_date(year):
    return '1/'+ year

list_of_datecolumns = ['LAST FUNDING DATE', 'FIRST FUNDING DATE', 'VALUATION DATE']

for column in list_of_datecolumns:
    df_filtered.loc[:,column] = df_filtered[column].astype(str)
    df_filtered.loc[:,column] = [parse_year_to_date(x) if len(x)==4 else x for x in df_filtered[column]]
    df_filtered.loc[:,column] = pd.to_datetime(df_filtered[column])

### 1.3  REVENUE as list

In [29]:
#function that takes in a value from a cell (string) and converts it to a list of numbers
def convert_to_list_of_numbers(cellValue):
    cellValueAsList = cellValue.split(';')
    return [float(x) if x != 'n/a' else 0 for x in cellValueAsList]

# Convert all cells of the "each round amount" column to a list with the function above
# if there is something to convert, otherwise create a list with just a 0 in it␣ ↪for that cell

df_filtered['REVENUE AS LIST'] = [convert_to_list_of_numbers(cell) if isinstance(cell, str) else [0] for cell in df_filtered['EACH ROUND AMOUNT']]

df_filtered['REVENUE AS LIST']

/tmp/ipykernel_238123/1660686734.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['REVENUE AS LIST'] = [convert_to_list_of_numbers(cell) if isinstance(cell, str) else [0] for cell in df_filtered['EACH ROUND AMOUNT']]


0                                  [0, 0, 0]
1                              [0, 1.7, 0.6]
2       [11.5, 0, 0.9, 2.3, 0.05, 2.5, 0.05]
3                                     [0, 0]
4                                        [0]
                        ...                 
8889                                     [0]
8890                                     [0]
8891                                     [0]
8892                                     [0]
8893                                     [0]
Name: REVENUE AS LIST, Length: 8698, dtype: object

In [30]:
#Function to check if Revenue List is 0. Maybe just add to the step before, but good for now.

def check_for_nulllist(cellValue):
    return (None if all(v == 0 for v in cellValue) is True else cellValue)

# To do: Get rid of apply and lambda, do it the easy way
df_filtered['REVENUE AS LIST'] = df_filtered['REVENUE AS LIST'].apply(lambda cell: None if len(cell)==1 else check_for_nulllist(cell))

#df_filtered['REVENUE AS LIST'].isnull().sum()

/tmp/ipykernel_238123/281430619.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['REVENUE AS LIST'] = df_filtered['REVENUE AS LIST'].apply(lambda cell: None if len(cell)==1 else check_for_nulllist(cell))


### 1.4 Make categories for VALUATION AVG value

NOTE: I thought this might help when we do evaluations

In [31]:
df_filtered['VALUATION CAT'] = np.nan

df_filtered.loc[df_filtered['VALUATION AVG'].between(0,2*10**6, inclusive = True), 'VALUATION CAT'] = '< 2 Mio.'
df_filtered.loc[df_filtered['VALUATION AVG'].between(2*10**6,10*10**6, inclusive = True), 'VALUATION CAT'] = '2-10 Mio.'
df_filtered.loc[df_filtered['VALUATION AVG'].between(10*10**6,50*10**6, inclusive = True), 'VALUATION CAT'] = '10-50 Mio.'
df_filtered.loc[df_filtered['VALUATION AVG'].between(50*10**6,100*10**6, inclusive = True), 'VALUATION CAT'] = '50-100 Mio.'

df_filtered['VALUATION CAT'].unique()
# There is one odd value of -92000000 in VALUATION column- need to check

/tmp/ipykernel_238123/4174041880.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['VALUATION CAT'] = np.nan
/tmp/ipykernel_238123/4174041880.py:3: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  df_filtered.loc[df_filtered['VALUATION AVG'].between(0,2*10**6, inclusive = True), 'VALUATION CAT'] = '< 2 Mio.'
/tmp/ipykernel_238123/4174041880.py:4: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  df_filtered.loc[df_filtered['VALUATION AVG'].between(2*10**6,10*10**6, inclusive = True), 'VALUATION CAT'] = '2-10 Mio.'
/tmp/ipykernel_238123/4174041880.py:5: FutureWarning: Boolean inputs to the `inclusive` argument are depr

array([nan, '2-10 Mio.', '10-50 Mio.', '50-100 Mio.', '< 2 Mio.'],
      dtype=object)

### 1.5 Make catageories for TOTAL FUNDING

## 2. Test Correlations

In [32]:
# Correlation between total funding and valuation. This is just a cheap test - 
#should be more accurate to choose total funding during seed stage for startups
#that are valuated in growth stage now

df_filtered['TOTAL FUNDING (USD M)'].corr(df_filtered['VALUATION AVG'])

0.08978403439298711

In [22]:
Result is 0.19 - so.. nope

SyntaxError: invalid syntax (1993949834.py, line 1)

## 3. Reducing the records to goal-records

Take care Svenja: Makes no sense for analytics to have limited records.

### 3.1 GROWTH STAGE

In [33]:
df_filtered['GROWTH STAGE'].unique()

array(['late growth stage', 'early growth stage', 'seed stage', nan,
       'not meaningful'], dtype=object)

We want to consider 'early growth stage', 'seed stage' and records without a value, since they still could be meaningful for us. There is only one record with a GROWTH STAGE of 'not meaningful', which can be neglected after manual review (too old). We delete all rows with neglected growth stage but keep records with unknown growth stage:


In [34]:
neglected_growth_stages = ['late growth stage', 'not meaningful']

indexrows = df_filtered[df_filtered['GROWTH STAGE'].isin(neglected_growth_stages)].index

df_filtered = df_filtered.drop(indexrows)
df_filtered.reset_index
df_filtered.shape

(7098, 49)

Reduced to 7098 records.

### 3.2 LAST ROUND

Reduces records which were last funded in the goal-growth stage or not funded at all. If last funding was later than A-series, the record is out.

In [35]:
df_filtered['LAST ROUND'].unique()

array(['ACQUISITION', 'SERIES A', nan, 'BUYOUT', 'ANGEL', 'SEED',
       'LATE VC', 'EARLY VC', 'GRANT', 'IPO', 'DEBT', 'MERGER',
       'PRIVATE PLACEMENT VC', 'LENDING CAPITAL', 'POST IPO DEBT',
       'SECONDARY', 'CONVERTIBLE', 'POST IPO EQUITY',
       'GROWTH EQUITY NON VC', 'GROWTH EQUITY VC', 'ICO',
       'POST IPO CONVERTIBLE', 'SERIES C', 'SERIES B', 'SERIES D',
       'SPINOUT', 'MEDIA FOR EQUITY', '-'], dtype=object)

In [36]:
neglected_last_rounds = ['ACQUISITION',
       'LATE VC', 'IPO', 'MERGER', 'POST IPO DEBT', 'POST IPO EQUITY',
       'GROWTH EQUITY NON VC', 'GROWTH EQUITY VC',
       'POST IPO CONVERTIBLE', 'SERIES C', 'SERIES B', 'SERIES D']

indexrowsLR = df_filtered[df_filtered['LAST ROUND'].isin(neglected_last_rounds)].index

df_filtered = df_filtered.drop(indexrowsLR)
df_filtered.reset_index
df_filtered.shape

(5877, 49)

Reduced to 5877.

### 3.3 Last Funding Year (< 01/2018)

In [37]:
# Sort out for last funding dates older than jan 2018
indexLFD = df_filtered[df_filtered['LAST FUNDING DATE'] < '2018-01-01'].index

df_filtered = df_filtered.drop(indexLFD)
df_filtered.reset_index
df_filtered.shape

#df_filtered['LAST FUNDING DATE'].unique()


(4086, 49)

Reduced to 4086.



In [38]:
#df_filtered['FOUNDERS GENDERS']= df_filtered['FOUNDERS GENDERS'].fillna('')
#df_filtered['FOUNDERS GENDERS'] = df_filtered['FOUNDERS GENDERS'].astype(str)

#df_filtered['FOUNDERS GENDERS'].unique()

#df_filtered['FOUNDERS GENDERS'].str.contains('female').sum()

In [39]:
#df_filtered.notnull().sum()

### 3.4 Neglect records with None Values in most important columns

Im Folgenden löschen wir records, die garkeine Jahresangaben haben und null revenue und keine Geschlechtsangabe.

In [40]:
#Check if LAST FUNDING DATE, FIRST FUNDING DATE, SEED YEAR, REVENUE AS LIST, VALUATION DATE IS ZERO, FOUNDERS GENDER

index_of_null_records = df_filtered[(df_filtered['LAST FUNDING DATE'].isnull() & df_filtered['FIRST FUNDING DATE'].isnull() & df_filtered['SEED YEAR'].isnull()& df_filtered['VALUATION DATE'].isnull() & df_filtered['REVENUE AS LIST'].isnull() & df_filtered['FOUNDERS GENDERS'].isnull())].index

df_filtered = df_filtered.drop(index_of_null_records)
df_filtered.reset_index
df_filtered.shape

(2354, 49)

Reduced to 2192 records.

## 4. Parse and sort by industry

## 4.3 Convert industries to list

In [41]:
def convert_industry_to_list(cellValue):
    industryAsList = cellValue.split(';')
    return None if pd.isnull(cellValue) else industryAsList

df_filtered['INDUSTRIES AS LIST'] = [None if pd.isnull(cell) else convert_industry_to_list(cell) for cell in df_filtered['INDUSTRIES']]

df_filtered['INDUSTRIES AS LIST']


2              [robotics]
6                    None
8                    None
9                [travel]
14            [education]
              ...        
8692             [health]
8696    [wellness beauty]
8700               [kids]
8742              [media]
8747                 None
Name: INDUSTRIES AS LIST, Length: 2354, dtype: object

### 4.2 Set main industry as first entry of industry list

In [42]:
df_filtered['MAIN INDUSTRY'] = [None if cell == None else cell[0] for cell in df_filtered['INDUSTRIES AS LIST']]

#df_filtered['MAIN INDUSTRY']

pd.set_option('display.max_rows',28)
df_filtered.groupby('MAIN INDUSTRY').size()

#df_filtered['MAIN INDUSTRY'].isnull().sum() = 161 Nicht kategorisierte Startups


MAIN INDUSTRY
dating                   5
education               74
energy                 201
enterprise software    143
event tech              10
fashion                 59
fintech                178
food                   155
gaming                  84
health                 445
home living             35
hosting                  7
jobs recruitment        46
kids                     6
legal                   39
marketing               82
media                  118
music                   28
real estate             80
robotics                65
security                58
semiconductors          35
sports                  42
telecom                 24
transportation          85
travel                  52
wellness beauty         37
dtype: int64

Save df_filtered dataframe:

In [43]:
%store df_filtered

Stored 'df_filtered' (DataFrame)




## Just blödsinn checking here for Education industry  

In [44]:
df_ed = df_filtered[df_filtered['MAIN INDUSTRY'] == 'education']

pd.set_option('display.max_rows',48)
df_ed.count()


NAME                   74
PROFILE URL            74
WEBSITE                74
HQ REGION              74
HQ COUNTRY             74
                       ..
VALUATION (EUR) AVG    42
REVENUE AS LIST        39
VALUATION CAT          42
INDUSTRIES AS LIST     74
MAIN INDUSTRY          74
Length: 51, dtype: int64

In [47]:
#df_ed.groupby('HQ CITY').size()

df_ed['FOUNDERS GENDERS'].str.contains('female').sum()
df_ed['FOUNDERS BACKGROUNDS'].unique()

df_ed['VALUATION'].unique

17